# Parse openai pricing

TODO: Need to finish the html page parser. Not getting all the tables!

In [4]:
from oa._resources import Resources

r = Resources()

html = r.pricing_page_html
len(html)

112759

In [5]:
from oa._resources import parse_pricing_page

t = parse_pricing_page(html)
list(t)


['Text tokens', 'Image generationPriceper/image']

In [ ]:
r.pricing_info  # makes (or loads) pricing info by parsing the html page

{'Text tokens': {'pricing_table_schema': ['Model',
   'Input',
   'Cached input',
   'Output'],
  'pricing_table': [{'Model': 'gpt-4.5-preview',
    'Alternate Model Names': ['gpt-4.5-preview-2025-02-27'],
    'Input': '$75.00',
    'Cached input': '$37.50',
    'Output': '$150.00'},
   {'Model': 'gpt-4o',
    'Alternate Model Names': ['gpt-4o-2024-08-06'],
    'Input': '$2.50',
    'Cached input': '$1.25',
    'Output': '$10.00'},
   {'Model': 'gpt-4o-audio-preview',
    'Alternate Model Names': ['gpt-4o-audio-preview-2024-12-17'],
    'Input': '$2.50',
    'Cached input': '-',
    'Output': '$10.00'},
   {'Model': 'gpt-4o-realtime-preview',
    'Alternate Model Names': ['gpt-4o-realtime-preview-2024-12-17'],
    'Input': '$5.00',
    'Cached input': '$2.50',
    'Output': '$20.00'},
   {'Model': 'gpt-4o-mini',
    'Alternate Model Names': ['gpt-4o-mini-2024-07-18'],
    'Input': '$0.15',
    'Cached input': '$0.075',
    'Output': '$0.60'},
   {'Model': 'gpt-4o-mini-audio-preview',
 

In [3]:
list(r.pricing_info)

['Text tokens', 'Image generationPriceper/image']

## Using tabled.get_tables_from_url - but not sufficient for full information.

This basically uses `pd.read_html`. But is not sufficient 

Here we have the tables, but:
* Some of the Model column values have two mentioned models, and they are contatenated (so hard to separate)
* The tables don't come with their headers (to know what they correspond to)
* There are two versions of prices: With or with out Batch API

So perhaps we need to parse the actual html instead of forwarding to `pd.read_html`

In [ ]:
import pandas as pd 
import requests
import tabled

tables_pd = tabled.get_tables_from_url(pricing_url, url_to_html=('chrome', dict(wait=10)))
len(tables_pd)

10

In [113]:
tables_pd[0]

,Model,Input,Cached input,Output
0,gpt-4.5-previewgpt-4.5-preview-2025-02-27,$75.00,$37.50,$150.00
1,gpt-4ogpt-4o-2024-08-06,$2.50,$1.25,$10.00
2,gpt-4o-audio-previewgpt-4o-audio-preview-2024-...,$2.50,-,$10.00
3,gpt-4o-realtime-previewgpt-4o-realtime-preview...,$5.00,$2.50,$20.00
4,gpt-4o-minigpt-4o-mini-2024-07-18,$0.15,$0.075,$0.60
5,gpt-4o-mini-audio-previewgpt-4o-mini-audio-pre...,$0.15,-,$0.60
6,gpt-4o-mini-realtime-previewgpt-4o-mini-realti...,$0.60,$0.30,$2.40
7,o1o1-2024-12-17,$15.00,$7.50,$60.00
8,o3-minio3-mini-2025-01-31,$1.10,$0.55,$4.40
9,o1-minio1-mini-2024-09-12,$1.10,$0.55,$4.40


In [116]:
tables_pd[1]

,Model,Input,Cached input,Output
0,gpt-4o-audio-previewgpt-4o-audio-preview-2024-...,$40.00,-,$80.00
1,gpt-4o-mini-audio-previewgpt-4o-mini-audio-pre...,$10.00,-,$20.00
2,gpt-4o-realtime-previewgpt-4o-realtime-preview...,$40.00,$2.50,$80.00
3,gpt-4o-mini-realtime-previewgpt-4o-mini-realti...,$10.00,$0.30,$20.00


## Using AI to parse the table

In [122]:
openai_api_pricing_schema_description_key = 'data/openai_api_pricing_schema_description.txt'
openai_api_pricing_schema_key = 'data/api_pricing_schema.json'
openai_api_pricing_info_from_ai_key = 'data/openai_api_pricing_info_from_ai.json'

In [80]:
import pathlib 
import os 

if not os.path.isfile(openai_api_pricing_schema_description_key):
    print(f"Saving a schema_description to {openai_api_pricing_schema_description_key}...")

    schema_description = """
    A schema to contain the pricing information for OpenAI APIs.
    The first level field should be the name category of API, (things like "Text Tokens", "Audio Tokens", "Fine Tuning", etc.)
    The value of the first level field should be a dictionary with the following fields:
    - "pricing_table_schema": A schema for the pricing table. That is, the list of the columns
    - "pricing_table": A list of dicts, corresponding to the table of pricing information for the API category. 
        For example, ```[{"Model": ..., "Input": ..., ...,}, {"Model": ..., "Input": ..., ...,}, ...]```
        Note that Sometimes there is more than one model name, so we should also have, in there, a field for "Alternate Model Names"
    - "extras": A dictionary with extra information that might be parsed out of each category. 
    For example, there's often some text that gives a description of the API category, and/or specifies "Price per 1M tokens" etc.
        
    Note that some of these tables have a "Batch API" version too. 
    In this case, there should be extra fields that have the same name as the fields above, but with "- Batch" appended to the name.
    For example, "Text Tokens - Batch", "Fine Tuning - Batch", etc.

    It is important to note: These first level fields are not determined in advance. 
    They are determined by the data that is scraped from the page.
    Therefore, these names should not be in the schema. 
    What should be in the schema is the fact that the data should be a JSON whose first 
    level field describes a category, and whose value specifies information about these category.
    """

    pathlib.Path(openai_api_pricing_schema_description_key).write_text(schema_description)
else:
    print(f"Loading schema_description from {openai_api_pricing_schema_description_key}...")
    schema_description = pathlib.Path(openai_api_pricing_schema_description_key).read_text()


Loading schema_description from data/openai_api_pricing_schema_description.txt...


In [81]:
import json, pathlib

if not os.path.isfile(openai_api_pricing_schema_key):
    import oa 

    print(f"Making and saving a schema to {openai_api_pricing_schema_key}...")

    schema = oa.infer_schema_from_verbal_description(schema_description)
    pathlib.Path(openai_api_pricing_schema_key).write_text(json.dumps(schema, indent=2))
else:
    schema = json.loads(pathlib.Path(openai_api_pricing_schema_key).read_text())

print(json.dumps(lkj.truncate_dict_values(schema), indent=2)[:500] + '\n...')

Making and saving a schema to data/api_pricing_schema.json...
{
  "name": "OpenAI_API_Pricing_Schema",
  "type": "object",
  "properties": {
    "patternProperties": {
      "^.+$": {
        "type": "object",
        "properties": {
          "pricing_table_schema": {
            "type": "object",
            "properties": {
              "columns": {
                "type": "array",
                "items": {
                  "type": "string"
                }
              }
            },
            "required": [
              "columns"
            ]
...


In [ ]:
import pathlib, json 

if not os.path.isfile(openai_api_pricing_info_from_ai_key):
    import oa 

    print(f"Prompting for pricing information and saving to {openai_api_pricing_info_from_ai_key}...")
    
    prompt = """
    Parse through this html and extract the pricing information for the OpenAI APIs.
    The pricing information should be structured according to the schema described below:

    """ + schema_description + """ 

    Here is the html to parse:


    {html}

    """

    assert 'html' in locals(), "Please load the html into the variable 'html' before running this cell."
    
    get_pricing_info = oa.prompt_json_function(prompt, schema)

    pricing_info = get_pricing_info(html)

    pathlib.Path(openai_api_pricing_info_from_ai_key).write_text(json.dumps(pricing_info, indent=2))
else:
    print(f"Loading pricing information from {openai_api_pricing_info_from_ai_key}...")
    pricing_info = json.loads(pathlib.Path(openai_api_pricing_info_from_ai_key).read_text())



Prompting for pricing information and saving to data/openai_api_pricing_info.json...


In [87]:
list(pricing_info)
t = pricing_info['OpenAI_API_Pricing_Schema']
list(t)

['Text Tokens',
 'Audio Tokens',
 'Fine Tuning',
 'Built-in Tools',
 'Moderation',
 'Image Generation',
 'Embeddings',
 'Other Models']

In [93]:
import pandas as pd
import dol 

tables = dol.add_ipython_key_completions(dol.wrap_kvs(
    pricing_info['OpenAI_API_Pricing_Schema'],
    value_decoder=lambda x: pd.DataFrame(x['pricing_table'])
))
list(tables)


['Text Tokens',
 'Audio Tokens',
 'Fine Tuning',
 'Built-in Tools',
 'Moderation',
 'Image Generation',
 'Embeddings',
 'Other Models']

In [ ]:
def get_models(tables):
    for name, table in tables.items():
        # print(name)
        for model in list(table.get('Model', [])):
            yield model
        for model in list(table.get('Alternate Model Names', [])):
            yield model
            
models = list(get_models(tables))

In [153]:
"""
Data Access class for OpenAI pricing information.
Provides a caching layer over various data sources and computations.
"""

from dataclasses import dataclass, field
import json
import pathlib
from functools import partial
from typing import Dict, Any, Optional, Callable, Union, List

import pandas as pd
import dol

from oa._resources import resources_dir


def _ensure_dir_exists(path: Union[str, pathlib.Path]) -> None:
    """Ensure the directory for the given path exists."""
    pathlib.Path(path).parent.mkdir(parents=True, exist_ok=True)


@dataclass
class Dacc:
    """
    Data Access class for OpenAI pricing information.
    
    This class manages the retrieval, computation, and caching of
    OpenAI pricing data through a chain of computations.
    
    >>> dacc = Dacc()
    >>> dacc.resources_dir  # doctest: +ELLIPSIS
    '...misc/data'
    """
    resources_dir: str = resources_dir
    pricing_url: str = 'https://platform.openai.com/docs/pricing'
    
    schema_description_key: str = 'openai_api_pricing_schema_description.txt'
    schema_key: str = 'api_pricing_schema.json'
    pricing_html_key: str = 'openai_api_pricing.html'
    pricing_info_key: str = 'openai_api_pricing_info.json'
    pricing_info_from_ai_key: str = 'openai_api_pricing_info_from_ai.json'
    
    # Dependencies that can be injected
    get_pricing_page_html: Optional[Callable[[], str]] = None
    infer_schema_from_verbal_description: Optional[Callable[[str], Dict[str, Any]]] = None
    prompt_json_function: Optional[Callable[[str, Dict[str, Any]], Callable[[str], Dict[str, Any]]]] = None
    
    def __post_init__(self):
        """Initialize stores and ensure dependencies are available."""
        # Set up storage
        _ensure_dir_exists(f"{self.resources_dir}/{self.schema_description_key}")
        
        self.json_store = dol.JsonFiles(self.resources_dir)
        self.text_store = dol.TextFiles(self.resources_dir)
        
        # Import dependencies if not provided
        if self.get_pricing_page_html is None:
            from oa._resources import get_pricing_page_html
            self.get_pricing_page_html = get_pricing_page_html
            
        if self.infer_schema_from_verbal_description is None or self.prompt_json_function is None:
            import oa
            
            if self.infer_schema_from_verbal_description is None:
                self.infer_schema_from_verbal_description = oa.infer_schema_from_verbal_description
                
            if self.prompt_json_function is None:
                self.prompt_json_function = oa.prompt_json_function
    
    @dol.cache_this(cache='text_store', key=lambda self: self.schema_description_key)
    def schema_description(self) -> str:
        """Generate and retrieve the schema description for OpenAI API pricing."""
        return """
        A schema to contain the pricing information for OpenAI APIs.
        The first level field should be the name category of API, (things like "Text Tokens", "Audio Tokens", "Fine Tuning", etc.)
        The value of the first level field should be a dictionary with the following fields:
        - "pricing_table_schema": A schema for the pricing table. That is, the list of the columns
        - "pricing_table": A list of dicts, corresponding to the table of pricing information for the API category. 
            For example, ```[{"Model": ..., "Input": ..., ...,}, {"Model": ..., "Input": ..., ...,}, ...]```
            Note that Sometimes there is more than one model name, so we should also have, in there, a field for "Alternate Model Names"
        - "extras": A dictionary with extra information that might be parsed out of each category. 
        For example, there's often some text that gives a description of the API category, and/or specifies "Price per 1M tokens" etc.
            
        Note that some of these tables have a "Batch API" version too. 
        In this case, there should be extra fields that have the same name as the fields above, but with "- Batch" appended to the name.
        For example, "Text Tokens - Batch", "Fine Tuning - Batch", etc.

        It is important to note: These first level fields are not determined in advance. 
        They are determined by the data that is scraped from the page.
        Therefore, these names should not be in the schema. 
        What should be in the schema is the fact that the data should be a JSON whose first 
        level field describes a category, and whose value specifies information about these category.
        """
    
    @dol.cache_this(cache='text_store', key=lambda self: self.pricing_html_key)
    def pricing_page_html(self) -> str:
        """Retrieve the HTML for the OpenAI pricing page."""
        return self.get_pricing_page_html()
    
    @dol.cache_this(cache='json_store', key=lambda self: self.schema_key) 
    def schema(self) -> Dict[str, Any]:
        """Generate or retrieve the schema for OpenAI API pricing."""
        return self.infer_schema_from_verbal_description(self.schema_description)
    
    @dol.cache_this(cache='json_store', key=lambda self: self.pricing_info_from_ai_key)
    def pricing_info(self) -> Dict[str, Any]:
        """Extract pricing information from the HTML using AI."""
        prompt = f"""
        Parse through this html and extract the pricing information for the OpenAI APIs.
        The pricing information should be structured according to the schema described below:

        {self.schema_description}

        Here is the html to parse:

        {{html}}
        """
        
        get_pricing_info = self.prompt_json_function(prompt, self.schema)
        return get_pricing_info(self.pricing_page_html)
    
    @property
    def pricing_tables(self) -> Dict[str, pd.DataFrame]:
        """Convert pricing tables to pandas DataFrames."""
        tables = self.pricing_info.get('OpenAI_API_Pricing_Schema', {})
        
        # Create a mapping interface that transforms table data to DataFrames
        return dol.add_ipython_key_completions(
            dol.wrap_kvs(
                tables,
                value_decoder=lambda x: pd.DataFrame(x.get('pricing_table', []))
            )
        )
    
    def list_pricing_categories(self) -> List[str]:
        """List available pricing categories."""
        return list(self.pricing_info.get('OpenAI_API_Pricing_Schema', {}))
    
    def get_pricing_table(self, category: str) -> pd.DataFrame:
        """Get a specific pricing table as a pandas DataFrame."""
        return self.pricing_tables[category]

In [ ]:
dacc = Dacc()
list(dacc.json_store)



['api_pricing_schema.json',
 'openai_api_pricing_schema_description.txt',
 'openai_api_pricing_info_from_ai.json',
 'openai_api_pricing_info.json',
 'openai_api_pricing.html']

'JsonFiles'

## ChatDacc: Demo

For background see this discussion comment: [Accessing OpenAI Chat Data and Shared Conversations
](https://github.com/thorwhalen/oa/discussions/11#discussioncomment-11852719).

In [164]:
from oa.chats import ChatDacc
from pprint import pprint
import pandas as pd

url = 'https://chatgpt.com/share/6788d539-0f2c-8013-9535-889bf344d7d5'
dacc = ChatDacc(url)

In [ ]:
pprint(dacc.basic_turns_data)

[{'content': 'This conversation is meant to be used as an example, for '
             'testing, and/or for figuring out how to parse the html and json '
             'of a conversation. \n'
             '\n'
             "As such, we'd like to keep it short. \n"
             '\n'
             'Just say "Hello World!" back to me for now, and then in a second '
             'line write 10 random words. ',
  'id': '3b469b70-b069-4640-98af-5417491bb626',
  'role': 'user',
  'time': 1737020650.866734},
 {'content': 'Hello World!  \n'
             'apple, river, galaxy, chair, melody, shadow, cloud, puzzle, '
             'flame, whisper.',
  'id': '38ee4a3f-8487-4b35-9f92-ddee57c25d0a',
  'role': 'assistant',
  'time': 1737020652.436654},
 {'content': 'Now, so we get code blocks (and a second user query), write a '
             'python code block containing a hello world print.',
  'id': '01808c08-dc33-4932-bc12-64bd7e936760',
  'role': 'user',
  'time': 1737020700.676552},
 {'content': '``

In [167]:
dacc.basic_turns_df

,id,role,content,time
0,3b469b70-b069-4640-98af-5417491bb626,user,This conversation is meant to be used as an ex...,2025-01-16 09:44:10
1,38ee4a3f-8487-4b35-9f92-ddee57c25d0a,assistant,"Hello World! \napple, river, galaxy, chair, m...",2025-01-16 09:44:12
2,01808c08-dc33-4932-bc12-64bd7e936760,user,"Now, so we get code blocks (and a second user ...",2025-01-16 09:45:00
3,be4486db-894f-4e6f-bd0a-22d9d2facf69,assistant,"```python\nprint(""Hello, World!"")\n```",2025-01-16 09:45:01


There's more turn keys than those that are shown here. 
The basic turns data just shows turns that have non-null roles and content. 

In [178]:
dacc.turns_data_keys

['adff303b-75cc-493c-b757-605adadb8e56',
 '1473f2d9-ba09-4cd7-90c4-1452898676de',
 '40c5ed53-2b82-4e38-a4ec-dabf8f589553',
 '3b469b70-b069-4640-98af-5417491bb626',
 '94469d89-ff37-48c1-bdab-44b27299d79b',
 '38ee4a3f-8487-4b35-9f92-ddee57c25d0a',
 '01808c08-dc33-4932-bc12-64bd7e936760',
 'be4486db-894f-4e6f-bd0a-22d9d2facf69']

In [203]:
pprint(dacc.turns_data[dacc.turns_data_keys[3]])

{'children': ['94469d89-ff37-48c1-bdab-44b27299d79b'],
 'id': '3b469b70-b069-4640-98af-5417491bb626',
 'message': {'author': {'metadata': {}, 'role': 'user'},
             'content': {'content_type': 'text',
                         'parts': ['This conversation is meant to be used as '
                                   'an example, for testing, and/or for '
                                   'figuring out how to parse the html and '
                                   'json of a conversation. \n'
                                   '\n'
                                   "As such, we'd like to keep it short. \n"
                                   '\n'
                                   'Just say "Hello World!" back to me for '
                                   'now, and then in a second line write 10 '
                                   'random words. ']},
             'create_time': 1737020650.866734,
             'id': '3b469b70-b069-4640-98af-5417491bb626',
             'metadata': 

You can get a list of all possible fields (up to the time of writing this, up to our knowledge).

In [220]:
next(iter(dacc.turns_data_ssot.values()))

{'adff303b-75cc-493c-b757-605adadb8e56': {'description': 'Unique identifier for the conversation turn.',
  'example': 'adff303b-75cc-493c-b757-605adadb8e56'},
 'id': {'description': 'The unique identifier for a message or turn in the conversation.',
  'example': 'adff303b-75cc-493c-b757-605adadb8e56'},
 'children': {'description': 'An array of identifiers for the child messages that are part of this turn.',
  'example': ['1473f2d9-ba09-4cd7-90c4-1452898676de']},
 'message': {'description': 'An object containing details about the message sent during this turn.',
  'example': {'id': '1473f2d9-ba09-4cd7-90c4-1452898676de',
   'author': {'role': 'system', 'metadata': {}},
   'content': {'content_type': 'text', 'parts': ['']},
   'status': 'finished_successfully',
   'end_turn': True,
   'weight': 0,
   'metadata': {'is_visually_hidden_from_conversation': True,
    'shared_conversation_id': '6788d539-0f2c-8013-9535-889bf344d7d5'},
   'recipient': 'all'}},
 'parent': {'description': 'The ide

In [226]:
pd.DataFrame(index=dacc.turns_data_ssot.keys(), data=dacc.turns_data_ssot.values())

,description,example
id,The unique identifier for a message or turn in...,adff303b-75cc-493c-b757-605adadb8e56
children,An array of identifiers for the child messages...,[1473f2d9-ba09-4cd7-90c4-1452898676de]
message,An object containing details about the message...,"{'id': '1473f2d9-ba09-4cd7-90c4-1452898676de',..."
parent,The identifier of the parent message in the co...,adff303b-75cc-493c-b757-605adadb8e56
create_time,Timestamp indicating when the message was crea...,1737020650.866734
author,An object containing details about the author ...,"{'role': 'system', 'metadata': {}}"
content_type,The type of content included in the message (e...,text
parts,An array containing the actual message content...,[Original custom instructions no longer availa...
status,"The status of the message processing (e.g., fi...",finished_successfully
end_turn,Indicates whether this message concludes the c...,True


You have a tool to extract data from the turns data:

In [209]:
t = dacc.extract_turns(
    {
        'id': 'id', 'role': 
        'message.author.role', 
        'weight': 'message.weight'
    }
)
pd.DataFrame(t)

,id,role,weight
0,adff303b-75cc-493c-b757-605adadb8e56,None,NaN
1,3b469b70-b069-4640-98af-5417491bb626,user,1.0
2,94469d89-ff37-48c1-bdab-44b27299d79b,assistant,1.0
3,38ee4a3f-8487-4b35-9f92-ddee57c25d0a,assistant,1.0
4,01808c08-dc33-4932-bc12-64bd7e936760,user,1.0
5,be4486db-894f-4e6f-bd0a-22d9d2facf69,assistant,1.0


And there's also `metadata` (which in the original json, contains `mappings` and l`inear_conversation` fields, which are two fields that contain the turns data (the `mapping` maps them by ids, the `linear_conversation` fields are just a list of the values))

In [194]:
dacc.metadata

{'title': 'Test Chat 1',
 'create_time': 1737020729.060687,
 'update_time': 1737020733.031014,
 'moderation_results': [],
 'current_node': 'be4486db-894f-4e6f-bd0a-22d9d2facf69',
 'conversation_id': '6788d539-0f2c-8013-9535-889bf344d7d5',
 'is_archived': False,
 'safe_urls': [],
 'default_model_slug': 'gpt-4o',
 'disabled_tool_ids': [],
 'is_public': True,
 'has_user_editable_context': False,
 'continue_conversation_url': 'https://chatgpt.com/share/6788d539-0f2c-8013-9535-889bf344d7d5/continue',
 'moderation_state': {'has_been_moderated': False,
  'has_been_blocked': False,
  'has_been_accepted': False,
  'has_been_auto_blocked': False,
  'has_been_auto_moderated': False},
 'is_indexable': False,
 'is_better_metatags_enabled': True}

In [227]:
pd.DataFrame(index=dacc.metadata_ssot.keys(), data=dacc.metadata_ssot.values())

,description,example
title,The title of the conversation.,Test Chat 1
create_time,The timestamp (in seconds since epoch) when th...,1737020729.060687
update_time,The timestamp (in seconds since epoch) when th...,1737020733.031014
moderation_results,An array that holds results from any moderatio...,[]
current_node,The unique identifier of the current node in t...,be4486db-894f-4e6f-bd0a-22d9d2facf69
conversation_id,A unique identifier for the entire conversation.,6788d539-0f2c-8013-9535-889bf344d7d5
is_archived,A boolean indicating whether the conversation ...,False
safe_urls,An array of URLs deemed safe within the contex...,[]
default_model_slug,The identifier for the default model used in t...,gpt-4o
disabled_tool_ids,An array of tool identifiers that have been di...,[]


And then, there's where all of this came from: The base json that was extracted from the conversations html:

In [195]:
list(dacc.full_json_dict)

['basename', 'future', 'isSpaMode', 'state']

This is a large nested json. The following is the path to the metadata (that contains the turns data etc.)

In [197]:
dacc.metadata_path

('state',
 'loaderData',
 'routes/share.$shareId.($action)',
 'serverResponse',
 'data')

In [200]:
from dol import path_get

data = path_get(dacc.full_json_dict, dacc.metadata_path)
list(data)

['title',
 'create_time',
 'update_time',
 'mapping',
 'moderation_results',
 'current_node',
 'conversation_id',
 'is_archived',
 'safe_urls',
 'default_model_slug',
 'disabled_tool_ids',
 'is_public',
 'linear_conversation',
 'has_user_editable_context',
 'continue_conversation_url',
 'moderation_state',
 'is_indexable',
 'is_better_metatags_enabled']

In [201]:
t = dacc.extract_turns(
    {'id': 'id', 'role': 'message.author.role', 'content': 'message.content.parts'}
)
list(t)

[{'id': 'adff303b-75cc-493c-b757-605adadb8e56', 'role': None, 'content': None},
 {'id': '3b469b70-b069-4640-98af-5417491bb626',
  'role': 'user',
  'content': ['This conversation is meant to be used as an example, for testing, and/or for figuring out how to parse the html and json of a conversation. \n\nAs such, we\'d like to keep it short. \n\nJust say "Hello World!" back to me for now, and then in a second line write 10 random words. ']},
 {'id': '94469d89-ff37-48c1-bdab-44b27299d79b',
  'role': 'assistant',
  'content': None},
 {'id': '38ee4a3f-8487-4b35-9f92-ddee57c25d0a',
  'role': 'assistant',
  'content': ['Hello World!  \napple, river, galaxy, chair, melody, shadow, cloud, puzzle, flame, whisper.']},
 {'id': '01808c08-dc33-4932-bc12-64bd7e936760',
  'role': 'user',
  'content': ['Now, so we get code blocks (and a second user query), write a python code block containing a hello world print.']},
 {'id': 'be4486db-894f-4e6f-bd0a-22d9d2facf69',
  'role': 'assistant',
  'content': [

In [163]:
dacc.turns_df

,role,content,time
id,,,
3b469b70-b069-4640-98af-5417491bb626,user,This conversation is meant to be used as an ex...,2025-01-16 09:44:10
38ee4a3f-8487-4b35-9f92-ddee57c25d0a,assistant,"Hello World! \napple, river, galaxy, chair, m...",2025-01-16 09:44:12
01808c08-dc33-4932-bc12-64bd7e936760,user,"Now, so we get code blocks (and a second user ...",2025-01-16 09:45:00
be4486db-894f-4e6f-bd0a-22d9d2facf69,assistant,"```python\nprint(""Hello, World!"")\n```",2025-01-16 09:45:01


## Bootstrapping the parsing (and maintaining the parser)

In [ ]:
from oa.chats import url_to_html, parse_shared_chatGPT_chat

url = 'https://chatgpt.com/share/6788d539-0f2c-8013-9535-889bf344d7d5'
chat_html = url_to_html(url)
chat_json_dict = parse_shared_chatGPT_chat(chat_html)
list(chat_json_dict)

['basename', 'future', 'isSpaMode', 'state']

If you have a look at the `parse_shared_chatGPT_chat` you'll notice it's parametrized by several arguments you didn't have to specify.

```python
def parse_shared_chatGPT_chat(
    html: str,
    is_target_string: Union[str, Callable] = 'data.*mapping.*message',
    *,
    variable_name: str = 'window.__remixContext',
    json_pattern: str = '(\\{.*?\\});') -> dict
) -> dict:
```

In [29]:
from i2 import Sig

print(',\n'.join(str(Sig(parse_shared_chatGPT_chat)).split(',')))

(html: str,
 is_target_string: Union[str,
 Callable] = 'data.*mapping.*message',
 *,
 variable_name: str = 'window.__remixContext',
 json_pattern: str = '(\\{.*?\\});') -> dict


We did this so that we can easily maintain the parser's definition. 

At the time of writing this, we've noticed that the json we want is assigned to a 
variable named `window.__remixContext` and has fields `data`, `mapping`, and `message` in that order. 

But to be able to get that in the first place, we used `parse_shared_chatGPT_chat` with different parameters. 

We tried it with a chat we knew the contents of, and asked `parse_shared_chatGPT_chat` to find the json 
based on a substring we knew should be in there:

In [32]:
from oa.chats import url_to_html, parse_shared_chatGPT_chat

url = 'https://chatgpt.com/share/6788d539-0f2c-8013-9535-889bf344d7d5'
string_contained_in_conversation = 'apple, river, galaxy, chair, melody, shadow, cloud, puzzle, flame, whisper'
chat_html = url_to_html(url)
chat_json_dict = parse_shared_chatGPT_chat(chat_html, is_target_string=string_contained_in_conversation)
list(chat_json_dict)

['basename', 'future', 'isSpaMode', 'state']

Let's get a truncated version of this dict, to make sure it's not too big to study...

In [21]:
from lkj import truncate_dict_values
from pprint import pprint
from pyperclip import copy


t = truncate_dict_values(chat_json_dict)
list(t)

['basename', 'future', 'isSpaMode', 'state']

In [22]:
# save this truncated dict to a file
import tempfile
import json
from pathlib import Path

tmp_filepath = tempfile.mktemp()
Path


len(json.dumps(t))

330325

## Parsing out the json

This is probably the most fragile part of the process. 
It's the part that broke previous tools I found to solve my problem.

The approach I'm taking here is to identify a part of the conversation that I know should be in the conversation, and using that to find what I'm looking for. 

So what I did here is copy the html of a conversation and used chatGPT (o1) to help me out. 
[Here's the conversation](https://chatgpt.com/share/6788d964-9e60-8013-a2b3-e191d567c8ad).

In [6]:
# Bootstrapping (when you don't know the structure, but you do know of a string that is contained in the conversation)

from oa.chats import parse_shared_chatGPT_chat, url_to_html

url = 'https://chatgpt.com/share/6788d539-0f2c-8013-9535-889bf344d7d5'
string_contained_in_conversation = 'apple, river, galaxy, chair, melody, shadow, cloud, puzzle, flame, whisper'

chat_html = url_to_html(url)
chat_json_dict = parse_shared_chatGPT_chat(chat_html, is_target_string=string_contained_in_conversation)
list(chat_json_dict)

['basename', 'future', 'isSpaMode', 'state']

In [33]:
# get a truncated version of the chat_json_dict (too big to study!)
from lkj import truncate_dict_values
import json 

truncated_json_dict = truncate_dict_values(chat_json_dict, max_list_size=1, max_string_size=90)
truncated_json_dict_string = json.dumps(truncated_json_dict, indent=2)
assert string_contained_in_conversation in truncated_json_dict_string, (
    "The string_contained_in_conversation is not in the truncated_json_dict_string. Increase the max_* parameters."
)

In [106]:
# Save to a file to go study it...

import tempfile
from pathlib import Path

tmp_filepath = tempfile.mktemp() + '.json'
Path(tmp_filepath).write_text(truncated_json_dict_string)
print(f"{len(truncated_json_dict_string)} characters written to {tmp_filepath}")

584063 characters written to /var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/tmph5v0uvws.json


## Parsing out the conversation part of the json

In [36]:
from oa.chats import find_all_matching_paths_in_list_values
from dol import path_get

paths = find_all_matching_paths_in_list_values(truncated_json_dict, target_value=string_contained_in_conversation)
path = next(paths)
path

('state',
 'loaderData',
 'routes/share.$shareId.($action)',
 'serverResponse',
 'data',
 'mapping',
 '38ee4a3f-8487-4b35-9f92-ddee57c25d0a',
 'message',
 'content',
 'parts')

So now we know the path to where our target string can be found:

In [37]:
path_get(truncated_json_dict, path)

['Hello World!  \napple, river, galaxy, chair, melody, shadow, cloud, puzzle, flame, whisper.']

We can also look around it to get a better sense of the relevant json structure

In [38]:
path_get(truncated_json_dict, path[:-1])

{'content_type': 'text',
 'parts': ['Hello World!  \napple, river, galaxy, chair, melody, shadow, cloud, puzzle, flame, whisper.']}

In [39]:
path_get(truncated_json_dict, path[:-2])

{'id': '38ee4a3f-8487-4b35-9f92-ddee57c25d0a',
 'author': {'role': 'assistant', 'metadata': {}},
 'create_time': 1737020652.436654,
 'content': {'content_type': 'text',
  'parts': ['Hello World!  \napple, river, galaxy, chair, melody, shadow, cloud, puzzle, flame, whisper.']},
 'status': 'finished_successfully',
 'end_turn': True,
 'weight': 1,
 'metadata': {'finish_details': {'type': 'stop', 'stop_tokens': [200002]},
  'is_complete': True,
  'citations': [],
  'content_references': [],
  'message_type': None,
  'model_slug': 'gpt-4o',
  'default_model_slug': 'gpt-4o',
  'parent_id': '94469d89-ff37-48c1-bdab-44b27299d79b',
  'request_id': '902d2a5a0ed1e15c-MRS',
  'timestamp_': 'absolute',
  'shared_conversation_id': '6788d539-0f2c-8013-9535-889bf344d7d5'},
 'recipient': 'all'}

In [87]:
print(path[:-4])

('state', 'loaderData', 'routes/share.$shareId.($action)', 'serverResponse', 'data', 'mapping')


A wild guess.
`('state', 'loaderData', 'routes/share.$shareId.($action)', 'serverResponse', 'data', 'mapping')` is where the "turns" of the conversation are.

In [40]:
turns_path = ('state', 'loaderData', 'routes/share.$shareId.($action)', 'serverResponse', 'data', 'mapping')
turns_data = path_get(truncated_json_dict, turns_path)
list(turns_data)

['adff303b-75cc-493c-b757-605adadb8e56',
 '1473f2d9-ba09-4cd7-90c4-1452898676de',
 '40c5ed53-2b82-4e38-a4ec-dabf8f589553',
 '3b469b70-b069-4640-98af-5417491bb626',
 '94469d89-ff37-48c1-bdab-44b27299d79b',
 '38ee4a3f-8487-4b35-9f92-ddee57c25d0a',
 '01808c08-dc33-4932-bc12-64bd7e936760',
 'be4486db-894f-4e6f-bd0a-22d9d2facf69']

In [41]:
chat_data_path = turns_path[:-1]
list(path_get(truncated_json_dict, chat_data_path))

['title',
 'create_time',
 'update_time',
 'mapping',
 'moderation_results',
 'current_node',
 'conversation_id',
 'is_archived',
 'safe_urls',
 'default_model_slug',
 'disabled_tool_ids',
 'is_public',
 'linear_conversation',
 'has_user_editable_context',
 'continue_conversation_url',
 'moderation_state',
 'is_indexable',
 'is_better_metatags_enabled']

In [42]:
metadata = path_get(truncated_json_dict, chat_data_path)
metadata = {k: v for k, v in metadata.items() if k != 'mapping'}
metadata

{'title': 'Test Chat 1',
 'create_time': 1737020729.060687,
 'update_time': 1737020733.031014,
 'moderation_results': [],
 'current_node': 'be4486db-894f-4e6f-bd0a-22d9d2facf69',
 'conversation_id': '6788d539-0f2c-8013-9535-889bf344d7d5',
 'is_archived': False,
 'safe_urls': [],
 'default_model_slug': 'gpt-4o',
 'disabled_tool_ids': [],
 'is_public': True,
 'linear_conversation': [{'id': 'adff303b-75cc-493c-b757-605adadb8e56',
   'children': ['1473f2d9-ba09-4cd7-90c4-1452898676de']}],
 'has_user_editable_context': False,
 'continue_conversation_url': 'https://chatgpt.com/share/6788d539-0f2c-8013-9535-889bf344d7d5/continue',
 'moderation_state': {'has_been_moderated': False,
  'has_been_blocked': False,
  'has_been_accepted': False,
  'has_been_auto_blocked': False,
  'has_been_auto_moderated': False},
 'is_indexable': False,
 'is_better_metatags_enabled': True}

## Make some documentation for this part of the chat json

In [34]:
from oa import prompt_json_function


mk_json_field_doc_ssot = prompt_json_function("""
You are a technical writer specialized in documenting JSON fields. 
Below is a JSON object. I'd like you to document each field, giving me a json
whose fields are the names of the fields and the values are the description and example.
That is: {field_name: {"description": ..., "example": ...}, ...}
                                              
The context is:
{context: just a general context}
                                              
Here's an example json object:

{example_json}
""")

mk_json_field_doc_ssot

<function oa.tools.prompt_function.<locals>.ask_oa(example_json, *, field_name=' {"description": ..., "example": ...}, ...', context=' just a general context')>

In [ ]:
turns_data_ssot = mk_json_field_doc_ssot(
    json.dumps(turns_data, indent=2), 
    context="The conversation 'turns' of the json object holding a shared chatGPT conversation."
)
pprint(turns_data_ssot['result'])

{'adff303b-75cc-493c-b757-605adadb8e56': {'description': 'Unique identifier '
                                                         'for the conversation '
                                                         'turn.',
                                          'example': 'adff303b-75cc-493c-b757-605adadb8e56'},
 'author': {'description': 'An object containing details about the author of '
                           'the message, including role and any additional '
                           'metadata.',
            'example': {'metadata': {}, 'role': 'system'}},
 'children': {'description': 'An array of identifiers for the child messages '
                             'that are part of this turn.',
              'example': ['1473f2d9-ba09-4cd7-90c4-1452898676de']},
 'citations': {'description': 'An array of citations (if any) included with '
                              'the message.',
               'example': []},
 'content_references': {'description': 'An array of content ref

In [ ]:
metadata_ssot = mk_json_field_doc_ssot(
    metadata,
    context="The metadata of the json object holding a shared chatGPT conversation."
)
pprint(metadata_ssot['result'])

{'continue_conversation_url': {'description': 'A URL that allows users to '
                                              'continue the conversation from '
                                              'its current state.',
                               'example': 'https://chatgpt.com/share/6788d539-0f2c-8013-9535-889bf344d7d5/continue'},
 'conversation_id': {'description': 'A unique identifier for the entire '
                                    'conversation.',
                     'example': '6788d539-0f2c-8013-9535-889bf344d7d5'},
 'create_time': {'description': 'The timestamp (in seconds since epoch) when '
                                'the conversation was created.',
                 'example': 1737020729.060687},
 'current_node': {'description': 'The unique identifier of the current node in '
                                 'the conversation flow.',
                  'example': 'be4486db-894f-4e6f-bd0a-22d9d2facf69'},
 'default_model_slug': {'description': 'The identifier f

In [98]:
from oa import prompt_function


mk_json_field_documentation = prompt_function("""
You are a technical writer specialized in documenting JSON fields. 
Below is a JSON object. I'd like you to document each field in a markdown table.
The table should contain the name, description, and example value of each field.
                                              
The context is:
{context: just a general context}
                                              
Here's an example json object:

{example_json}
""")

mk_json_field_documentation

<function oa.tools.prompt_function.<locals>.ask_oa(example_json, *, context=' just a general context')>

In [110]:
turn_data_docs = mk_json_field_documentation(
    json.dumps(turns_data, indent=2), 
    context="the conversation 'turns' of the json object holding a shared chatGPT conversation"
)
print(turn_data_docs)

Here's the documentation for the provided JSON object, structured in a markdown table format:

| Field Name                                         | Description                                                                                     | Example Value                                                                         |
|---------------------------------------------------|-------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------|
| `<Conversation ID>`                               | Unique identifier for each turn in the conversation.                                          | `adff303b-75cc-493c-b757-605adadb8e56`                                             |
| `id`                                              | The unique identifier for the specific message/turn.                                         | `adff303b-75cc-493c-b757-605adadb8e56`   

In [99]:
metadata_field_docs = mk_json_field_documentation(
    json.dumps(metadata, indent=2), 
    context="metadata of a shared chatGPT conversation"
)
print(metadata_field_docs)

Here's the documentation for each field in the provided JSON object, formatted as a markdown table:

| Field Name                                | Description                                                                                          | Example Value                                                      |
|-------------------------------------------|------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------|
| `title`                                   | The title of the shared chat GPT conversation.                                                      | "Test Chat 1"                                                    |
| `create_time`                             | The timestamp when the conversation was created, represented in seconds since the Unix epoch.      | 1737020729.060687                                                |
| `update_time`                    

In [12]:
import re
import json
from typing import Callable, Optional

def raise_run_time_error(msg: str):
    raise RuntimeError(msg)

def extract_json_dict(
    string: str,
    object_filter: Callable,
    *,
    decoder: json.JSONDecoder = json.JSONDecoder(),
    not_found_callback: Optional[
        Callable
    ] = lambda string, object_filter: raise_run_time_error(
        "Object not found in string"
    ),
) -> dict:
    """
    Searches `string` from the beginning, attempting to decode consecutive
    JSON objects using `decoder.raw_decode`. When an object satisfies
    `object_filter`, returns it as a Python dictionary.
    """
    pos = 0
    while pos < len(string):
        try:
            obj, pos = decoder.raw_decode(string, pos)
            if object_filter(obj):
                return obj
        except json.JSONDecodeError:
            pos += 1
    return not_found_callback(string, object_filter)

def parse_shared_chatGPT_chat(
    html: str, 
    string_contained_in_conversation: str
) -> dict:
    """
    Locates the big JSON structure assigned to window.__remixContext, 
    checks that the conversation portion includes `string_contained_in_conversation`,
    and returns the JSON as a dict.
    """
    # Regex pattern to capture the object assigned to window.__remixContext = {...};
    pattern = r"window\.__remixContext\s*=\s*(\{.*?\});"
    match = re.search(pattern, html, flags=re.DOTALL)
    if not match:
        raise RuntimeError("Could not locate the JSON assigned to window.__remixContext")

    # Extract the raw JSON text (removing the trailing semicolon if needed)
    raw_json_text = match.group(1).strip()

    # We'll define a filter that checks if the conversation data includes the target string.
    # Because the JSON is quite large, we can simply check if the substring is present in the raw text:
    # but if we want to be more precise, we can parse first and only return if there's conversation data.
    def conversation_filter(obj):
        # Convert to string once more, or deeply check "routes/share.$shareId.($action)" for the substring.
        # For simplicity, do a textual check on the entire string representation. 
        # If needed, we can refine to parse a specific location in obj.
        return string_contained_in_conversation in json.dumps(obj, ensure_ascii=False)

    # Now parse the JSON, returning only if the filter passes
    extracted_dict = extract_json_dict(
        raw_json_text,
        object_filter=conversation_filter,
    )

    return extracted_dict


t = parse_shared_chatGPT_chat(html, existing_substring)

['basename', 'future', 'isSpaMode', 'state']